In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI, AuthenticationError

In [ ]:
load_dotenv()

In [ ]:
api_key = os.environ["OPENAI_KEY"]

def create_client(api_key):
    try:
        client = OpenAI(api_key=api_key)
        client.models.list()
        return client
    except AuthenticationError:
        print("Incorrect API")
    return None

client = create_client(api_key)

In [ ]:
def agent(prompt, context=[]):
    # łączymy historię konwersacji z nowym promptem
    messages = context + [{"role": "user", "content": prompt}]

    # wywołanie z MCP narzędziem DeepWiki
    response = client.responses.create(
        model="gpt-5",
        tools=[
            {
                "type": "mcp",
                "server_label": "deepwiki",
                "server_description": "A MCP server that allows access to structured wiki-like knowledge.",
                "server_url": "https://mcp.deepwiki.com/mcp",
                "require_approval": {
                    "never": {
                        "tool_names": ["ask_question", "read_wiki_structure"]
                    }
                }
            }
        ],
        input=prompt
    )

    # wynik tekstowy
    return response.output_text

In [ ]:
history = []

print("💬 MCP DeepWiki agent (type 'exit' to quit)\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    reply = agent(user_input, history)
    print("Agent:", reply)

    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": reply})